# Exercise - Cross-Validation and the Train-Test Split

In this exercise, you will apply what you have learned about splitting the dataset into training and test sets, impute missing values and scale features the correct way to avoid data leakage, and perform k-fold cross-validation in order to get more reliable and representative estimates of the model's performance on unseen data.  

The dataset is a modified version of the ["Housing Prices Dataset" from Kaggle](https://www.kaggle.com/datasets/yasserh/housing-prices-dataset).

In [26]:
# DO NOT MODIFY - imports
import pandas as pd

## 1. Data Preparation

Other than a few missing values which were introduced intentionally for the purpose of this demo, the dataset is clean and free from duplicated rows and other issues. You do not need to write your own code in this section. However, please read this section and inspect the code thoroughly to understand how the dataset is being set up for the next step.

In [27]:
# DO NOT MODIFY - Data loading and inspection
df = pd.read_csv("Housing_Modified_2.csv")
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420.0,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960.0,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960.0,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500.0,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420.0,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [28]:
# DO NOT MODIFY - Check for missing values
df.isnull().sum()

price                0
area                13
bedrooms             0
bathrooms            0
stories              0
mainroad             0
guestroom            0
basement             0
hotwaterheating      0
airconditioning      0
parking              0
prefarea             0
furnishingstatus     0
dtype: int64

We will impute the missing values in the `area` column.  
But first, run the cell below to convert the categorical "`yes`/`no`" columns to ones and zeros (integers).

In [29]:
# DO NOT MODIFY - Data preparation
# Convert "yes" and "no" to 1 and 0
yes_no_columns = ["mainroad", "guestroom", "basement", "hotwaterheating", "airconditioning", "prefarea"]
df[yes_no_columns] = df[yes_no_columns].map({"yes": 1, "no": 0}.get)
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420.0,4,2,3,1,0,0,0,1,2,1,furnished
1,12250000,8960.0,4,4,4,1,0,0,0,1,3,0,furnished
2,12250000,9960.0,3,2,2,1,0,1,0,0,2,1,semi-furnished
3,12215000,7500.0,4,2,2,1,0,1,0,1,3,1,furnished
4,11410000,7420.0,4,1,2,1,1,1,0,1,2,0,furnished


In [30]:
df.dtypes

price                 int64
area                float64
bedrooms              int64
bathrooms             int64
stories               int64
mainroad              int64
guestroom             int64
basement              int64
hotwaterheating       int64
airconditioning       int64
parking               int64
prefarea              int64
furnishingstatus     object
dtype: object

Run the cell below to one-hot-encode the `furnishingstatus` column with the first resulting column (`furnished`) dropped to avoid multicollinearity.

In [31]:
# DO NOT MODIFY - One-hot encoding `furnishingstatus`
df = pd.get_dummies(df, columns=["furnishingstatus"], drop_first=True)
df.dtypes

price                                int64
area                               float64
bedrooms                             int64
bathrooms                            int64
stories                              int64
mainroad                             int64
guestroom                            int64
basement                             int64
hotwaterheating                      int64
airconditioning                      int64
parking                              int64
prefarea                             int64
furnishingstatus_semi-furnished       bool
furnishingstatus_unfurnished          bool
dtype: object

We are now ready to split the data, impute missing values and scale the features if need be.

## 2. Train-Test Split and Proper Imputation and Scaling

Create the feature set, the matrix `X`, consisting of all columns but `price`. Then create the target, the array `y`, comprised of the values in the `price` column.

In [32]:
# FILL IN - Create feature set `X` and target `y`
X = df.drop(columns=["price"])
y = df["price"]

Split the data into training and testing sets using a 70/30 split. Shuffle the data while you split it, using a random seed of 52.

In [33]:
# DO NOT MODIFY - imports
from sklearn.model_selection import train_test_split

# FILL IN - Split the data into training and testing sets (70% train, 30% test) with a random state of 52
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=52)

Impute missing values in the `area` column using the `SimpleImputer` class from Scikit-Learn. Use the `median` strategy.

In [34]:
# DO NOT MODIFY - imports
from sklearn.impute import SimpleImputer

# FILL IN - Fit the imputer on the training data, then transform the training AND test data using the fitted imputer
imputer = SimpleImputer(strategy="median")
imputer.fit(X_train)
X_train_imputed = imputer.transform(X_train)
X_test_imputed = imputer.transform(X_test)

Below, we pick out columns of data that were originally numeric (and not just 0 or 1). Scale these features using a MinMaxScaler the correct way. - **HINT:** Only pass `X_train[numeric_columns]` and `X_test[numeric_columns]`, not all columns.

In [35]:
#  DO NOT MODIFY - Features that were originally numeric (and not just 0 or 1)
numeric_columns = ["area", "bedrooms", "bathrooms", "stories", "parking"] # bathrooms

# DO NOT MODIFY - imports
from sklearn.preprocessing import MinMaxScaler

# FILL IN - Fit the MinMaxScaler on the training data, then transform the training AND test data using the fitted scaler
# X_train_imputed and X_test_imputed are numpy arrays, so we need to convert them back to DataFrames to use column names
X_train_imputed_df = pd.DataFrame(X_train_imputed, columns=X_train.columns, index=X_train.index)
X_test_imputed_df = pd.DataFrame(X_test_imputed, columns=X_test.columns, index=X_test.index)

scaler = MinMaxScaler()
scaler.fit(X_train_imputed_df[numeric_columns])
X_train_scaled = X_train_imputed_df.copy()
X_test_scaled = X_test_imputed_df.copy()
X_train_scaled[numeric_columns] = scaler.transform(X_train_imputed_df[numeric_columns])
X_test_scaled[numeric_columns] = scaler.transform(X_test_imputed_df[numeric_columns])

Using `describe()`, verify that all values in both sets are between zero and one now.

In [44]:
# FILL IN - `describe()` the training set
print(X_train_scaled[numeric_columns].describe())

             area    bedrooms   bathrooms     stories     parking
count  381.000000  381.000000  381.000000  381.000000  381.000000
mean     0.239572    0.239501    0.133858    0.262467    0.225722
std      0.144209    0.182872    0.241559    0.280861    0.286791
min      0.000000    0.000000    0.000000    0.000000    0.000000
25%      0.134021    0.000000    0.000000    0.000000    0.000000
50%      0.205498    0.250000    0.000000    0.333333    0.000000
75%      0.305842    0.250000    0.000000    0.333333    0.333333
max      1.000000    1.000000    1.000000    1.000000    1.000000


In [37]:
# FILL IN - `describe()` the test set
# Note: Values outside [0, 1] (e.g., bathrooms max=1.5, bedrooms min=-0.25) can occur if test set contains values outside the range seen in the training set.
# This is normal for MinMaxScaler and not a bug.
print(X_test_scaled[numeric_columns].describe())

             area    bedrooms   bathrooms     stories     parking
count  164.000000  164.000000  164.000000  164.000000  164.000000
mean     0.245309    0.245427    0.164634    0.282520    0.243902
std      0.156820    0.188782    0.271949    0.308026    0.288610
min      0.024055   -0.250000    0.000000    0.000000    0.000000
25%      0.127148    0.000000    0.000000    0.000000    0.000000
50%      0.201375    0.250000    0.000000    0.333333    0.000000
75%      0.336082    0.250000    0.500000    0.333333    0.333333
max      0.958763    0.750000    1.500000    1.000000    1.000000


## 3. K-Fold Cross-Validation

Train a linear regression model on the training set and output its *training* score (which, by default, is the R-squared for regression tasks). - **HINT:** Use the `score()` method of the fitted model.

In [40]:
# DO NOT MODIFY - imports
from sklearn.linear_model import LinearRegression

# FILL IN - Train a linear regression model and output its R-squared score on the training set
model = LinearRegression()
model.fit(X_train_scaled, y_train)
train_score = model.score(X_train_scaled, y_train)
print(f"Training R-squared score: {train_score:.4f}")


Training R-squared score: 0.6829


Can we expect a similarly high score on unseen data? Before looking at the holdout (test) set, cross-validate the model using 5-fold CV and output the average score.

In [42]:
# DO NOT MODIFY - imports
from sklearn.model_selection import cross_val_score

# FILL IN - Cross-validate the model using 5-fold CV and output the mean R-squared score
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2')
mean_cv_score = cv_scores.mean()
print(f"Mean cross-validated R-squared score: {mean_cv_score:.4f}")

Mean cross-validated R-squared score: 0.6365


Finally, evaluate the trained model on the test set and output the test score (R-squared). Is it closer to the training score or the average CV score?

In [43]:
# DO NOT MODIFY - imports
from sklearn.metrics import r2_score

# FILL IN - Evaluate the model on the test set and output its R-squared score
y_pred = model.predict(X_test_scaled)
test_score = r2_score(y_test, y_pred)
print(f"Test R-squared score: {test_score:.4f}")

Test R-squared score: 0.6559
